* [Haskell implementation](http://www.danielbrice.net/blog/10/)
* [Better haskell implementation](https://gist.github.com/tbepler/e776b74715705ad5cac6)
* [Doubly Recursive Multivariate Automatic Differentiation](http://www1.american.edu/cas/mathstat/People/kalman/pdffiles/mmgautodiff.pdf)

Doubly recursive. What does this mean here? So we are recursing through the higher order derivatives, as well as the variables?

![pic](_Images/RecursiveDuals.png)

In [18]:
-- define Dual numbers to be a pair of numbers
data Dual = Dual  {x :: Float, dx :: Float}  deriving (Show,Eq)
-- overload operators/define algebra
instance Num Dual where -- notation? dx or x'
    (Dual x x') + (Dual y y') = Dual (x + y) (x' + y') -- sum rule
    (Dual x x') - (Dual y y') = Dual (x - y) (x' - y') 
    (Dual x x') * (Dual y y') = Dual (x * y) (x * y' + x' * y) -- product rule
instance Fractional Dual where
    (Dual x x') / (Dual y y') = Dual (x / y) ((y * y' - x * x') / y*y) -- quotient rule
    fromRational x = Dual (fromRational x) 0
-- test
let f x1 x2 = x1*x2 in f(Dual 4 1)(Dual 2 0)

Dual {x = 8.0, dx = 2.0}

In [19]:
-- overload elementary functions 
instance Floating Dual where 
    sin (Dual x x') = Dual (sin x) (x' * cos x) 
    cos (Dual x x') = Dual (cos x) (x' * (-(sin x))) 
    exp (Dual x x') = Dual (exp x) (x' * exp x)
-- test
let f x1 x2 = x2 * sin((x1 * x1) + 3.0) in f(Dual 5 1)(Dual 2 0) 

Dual {x = 0.5418116, dx = -19.252117}

In [47]:
dx (Dual 5 (Dual 2 3))

Couldn't match expected type ‘Float’ with actual type ‘Dual’
In the second argument of ‘Dual’, namely ‘(Dual 2 3)’
In the first argument of ‘dx’, namely ‘(Dual 5 (Dual 2 3))’

In [53]:
--DS(n,m) = DS(n−1,m) × DS(n,m−1)
data Tree a = Leaf a | Node a (Tree a) (Tree a) deriving (Show, Eq)

ds :: Integral a =>  a -> a -> Tree a
ds 0 _ = Leaf 1
ds _ 0 = Leaf 1
ds n m = Node (n+m) (ds (n-1) m) (ds n (m-1))

ds 2 3

Node 5 (Node 4 (Leaf 1) (Node 3 (Leaf 1) (Node 2 (Leaf 1) (Leaf 1)))) (Node 4 (Node 3 (Leaf 1) (Node 2 (Leaf 1) (Leaf 1))) (Node 3 (Node 2 (Leaf 1) (Leaf 1)) (Leaf 1)))

A new definition to do what dual numbers do in 1D with 1st order derivatives. 
$DS(n,m) = DS(n−1,m) × DS(n,m−1)$


i.e. if $f(x) \rightarrow (f(x), \frac{df}{dx})$, where $f(x) = DS(n-1,m) = DS(0,1)$ and $DS(n,m-1) = DS(1,0)= \frac{df}{dx}$

hmm. notation doesnt make sense?!?!?

In [46]:
--DS(n,m) = DS(n−1,m) × DS(n,m−1)
data Tree a = Leaf a | Node a (Tree a) (Tree a) deriving (Show, Eq)

ds :: Integral a => Dual -> a -> a -> Tree Dual
ds d 0 _ = Leaf (dx d)
ds d _ 0 = Leaf (dx d)
ds d n m = Node (Dual (dx d) 1) (ds (Dual (dx d) 1) (n-1) m) (ds (Dual (dx d) 1) n (m-1))

--ds (Dual 3 4) 2 3

Couldn't match expected type ‘Dual’ with actual type ‘Float’
In the first argument of ‘interactive:Ghci504.Leaf’, namely ‘(dx d)’
In the expression: interactive:Ghci504.Leaf (dx d)

Couldn't match expected type ‘Dual’ with actual type ‘Float’
In the first argument of ‘interactive:Ghci504.Leaf’, namely ‘(dx d)’
In the expression: interactive:Ghci504.Leaf (dx d)

In [ ]:
add :: DS -> DS -> DS
add x y
    if var x == 0 then
    if ord y == 0 then
    otherwise add (get x) ()


op_map :: (DS->DS) -> DS -> DS -> DS -- pass a handle to the function